In [1]:
import h5py
import scipy.io as io
import PIL.Image as Image
import numpy as np
import os
import glob
from matplotlib import pyplot as plt
from scipy.ndimage.filters import gaussian_filter 
import scipy
import json
from matplotlib import cm as CM
import time
from datetime import datetime


%matplotlib inline

In [2]:
shtb_path = os.path.join("datasets", "shtb")

shtb_train_images_path = os.path.join(shtb_path, "train_data", "images")
shtb_train_gt_path = os.path.join(shtb_path, "train_data", "ground-truth")
shtb_train_density_path = os.path.join(shtb_path, "train_data", "gt-density")
if not os.path.exists(shtb_train_density_path):
    os.mkdir(shtb_train_density_path)
    
shtb_test_images_path = os.path.join(shtb_path, "test_data", "images")
shtb_test_gt_path = os.path.join(shtb_path, "test_data", "ground-truth")
shtb_test_density_path = os.path.join(shtb_path, "test_data", "gt-density")
if not os.path.exists(shtb_test_density_path):
    os.mkdir(shtb_test_density_path)

In [3]:
def generate_density(img, mat, sigma):
    """ Given an image, its corresponding ground-truth annotated file, and a standard deviation, creates the ground-truth density map.
        img: The image corresponding to the ground-truth annotations
        mat: The coordinates of the annotations.
        sigma: A constant sigma used for the gaussian filter."""
    
    h, w, _ = img.shape
    k = np.zeros((h, w))
    
    gt_points = mat["image_info"][0,0][0,0][0]
    for (x, y) in gt_points.astype(int):
        if x < w and y < h:
            k[y, x] = 1  # Note the order of x and y here. Height is stored in first dimension
        else:
            print("This should never happen!")  # This would mean a head is annotated outside the image.
    
    density = scipy.ndimage.filters.gaussian_filter(k, sigma, mode='constant')  
    return density

In [4]:
def gen_all_densities(gt_source_path, image_source_path, density_save_path, sigma):
    """ Looks for all files in the gt_source_path directory. For each file, retrieve the corresponding image file and generate a ground-truth 
        density map. This density map is saved in the given density_save_path. A constant sigma is used for the gaussian filter for all images."""
    
    files = os.listdir(gt_source_path)
    n_files = len(files)
    
    now = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    info_file_path = os.path.join(density_save_path, "info.txt")
    with open(info_file_path, 'w') as f:
        f.write(f"Densities generated with a constant sigma of {sigma}.\n")
        f.write(f"gt source dir: {gt_source_path}, image source dir: {image_source_path}, save dir: {density_save_path}\n")
        f.write(f"Generated on {now}\n")

    print("Generating ground truth density maps.")
    start_time = time.time()
    for i, gt_file in enumerate(files):
        img_file = gt_file[3:].replace(".mat", ".jpg")
        density_file = gt_file.replace(".mat", ".h5")

        gt_file_path = os.path.join(gt_source_path, gt_file)
        img_file_path = os.path.join(image_source_path, img_file)
        density_file_path = os.path.join(density_save_path, density_file)

        mat = io.loadmat(gt_file_path)
        img = plt.imread(img_file_path)

        density = generate_density(img, mat, sigma)

        with  h5py.File(density_file_path, 'w') as f:
            f['density'] = density

        seconds_passed = time.time() - start_time
        if seconds_passed > 1:
            start_time = time.time()
            print(f"{i + 1} of {n_files} density maps created.")
              
    print(f"All {n_files} density maps created.")

In [5]:
# Set sigma to a desired value
sigma = 3

In [6]:
gen_all_densities(shtb_train_gt_path, shtb_train_images_path, shtb_train_density_path, sigma)  # Generates density maps for the training data
gen_all_densities(shtb_test_gt_path, shtb_test_images_path, shtb_test_density_path, sigma)  # Generates density maps for the test data

Generating ground truth density maps.
22 of 400 density maps created.
45 of 400 density maps created.
68 of 400 density maps created.
90 of 400 density maps created.
113 of 400 density maps created.
133 of 400 density maps created.
154 of 400 density maps created.
176 of 400 density maps created.
199 of 400 density maps created.
221 of 400 density maps created.
243 of 400 density maps created.
266 of 400 density maps created.
289 of 400 density maps created.
312 of 400 density maps created.
335 of 400 density maps created.
358 of 400 density maps created.
381 of 400 density maps created.
All 400 density maps created.
Generating ground truth density maps.
23 of 316 density maps created.
46 of 316 density maps created.
69 of 316 density maps created.
91 of 316 density maps created.
112 of 316 density maps created.
132 of 316 density maps created.
152 of 316 density maps created.
173 of 316 density maps created.
194 of 316 density maps created.
214 of 316 density maps created.
234 of 316 